In [17]:
import torch 
import torch.nn as nn
import torchvision
import numpy as np
import torchvision.transforms as transforms
from torchsummary import summary

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.003

In [4]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [15]:
class BiRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                           batch_first=True, bidirectional=True)
        # 2 for bidirection
        self.fc = nn.Linear(hidden_size * 2, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0),
                         self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0),
                         self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode the hidden state of the (-1)last time step
        out = self.fc(out[:, -1, :])
        return out
        

In [20]:
model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device)
module_parameters = filter(lambda p: p.requires_grad, model.parameters())
for p in module_parameters:
    print(p.size())
param_cnt = sum([np.prod(p.size()) for p in module_parameters])
print('total trainable params: %d' % param_cnt)
# lstm_l_1 28 * 128 * 4 + 128 * 4 = 14336 + 512 = 14848
# lstm_l_2 128 * 128 * 4 + 128 * 4 = 65536 + 512 = 66048
# lstm_2_1 128 * 128 * 4 + 128 * 4
# lstm 2_2 128 * 128 * 4 + 128 * 4
# fc 10*128 + 128 = 
print(model)



torch.Size([512, 28])
torch.Size([512, 128])
torch.Size([512])
torch.Size([512])
torch.Size([512, 28])
torch.Size([512, 128])
torch.Size([512])
torch.Size([512])
torch.Size([512, 256])
torch.Size([512, 128])
torch.Size([512])
torch.Size([512])
torch.Size([512, 256])
torch.Size([512, 128])
torch.Size([512])
torch.Size([512])
torch.Size([10, 256])
torch.Size([10])
total trainable params: 0
BiRNN(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=10, bias=True)
)


In [21]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/2], Step [100/600], Loss: 0.4213
Epoch [1/2], Step [200/600], Loss: 0.1349
Epoch [1/2], Step [300/600], Loss: 0.2646
Epoch [1/2], Step [400/600], Loss: 0.0775
Epoch [1/2], Step [500/600], Loss: 0.2080
Epoch [1/2], Step [600/600], Loss: 0.2139
Epoch [2/2], Step [100/600], Loss: 0.0477
Epoch [2/2], Step [200/600], Loss: 0.0388
Epoch [2/2], Step [300/600], Loss: 0.1379
Epoch [2/2], Step [400/600], Loss: 0.0750
Epoch [2/2], Step [500/600], Loss: 0.0515
Epoch [2/2], Step [600/600], Loss: 0.1083


In [23]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 97.47 %


In [ ]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')